In [8]:
!pip -q install fastapi uvicorn pyngrok sentence-transformers pymupdf faiss-cpu "transformers>=4.43" "accelerate>=0.33" "bitsandbytes>=0.43" "autoawq>=0.2.7" torch --extra-index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 88.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 90.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.3 MB/s eta 0:00:00:00:0100:01


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name="mistralai/Mistral-Nemo-Instruct-2407"
tokenizer=AutoTokenizer.from_pretrained(model_name, use_fast=True)
bnb_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
model=AutoModelForCausalLM.from_pretrained(model_name,
                                          quantization_config=bnb_config,
                                           device_map="auto").eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2026-01-18 14:52:48.640057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768747969.092915      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768747969.218697      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768747970.250629      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768747970.250660      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768747970.250663      55 computation_placer.cc:177] computation placer alr

...

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [10]:
save_dir = "mistral_nemo_instruct_2407_4bit_bnb"
try:
    model.save_pretrained(save_dir, safe_serialization=True)
    tokenizer.save_pretrained(save_dir)
    print("Saved quantized model to:", save_dir)
except Exception as e:
    print("Save 4-bit not supported in this env:", e)

Saved quantized model to: mistral_nemo_instruct_2407_4bit_bnb


In [1]:
import time
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

save_dir = "/kaggle/working/mistral_nemo_instruct_2407_4bit_bnb"

def load_model_and_tokenizer(path_or_id):
    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    tok = AutoTokenizer.from_pretrained(path_or_id, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        path_or_id,
        quantization_config=bnb_cfg,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
    return tok, model

In [2]:
if Path(save_dir).exists():
    print(f"Loading local 4-bit checkpoint: {save_dir}")
    tok, model = load_model_and_tokenizer(save_dir)
else:
    print("Local 4-bit checkpoint not found (saving likely not supported in this env).")

def generate_text(prompt, max_new_tokens=128, temperature=0.2, top_p=0.95):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    t0 = time.time()
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tok.eos_token_id,
        )
    t1 = time.time()
    text = tok.decode(out[0], skip_special_tokens=True)
    print(f"\n--- Generation ({t1 - t0:.2f}s) ---\n{text}\n")
    return text


Loading local 4-bit checkpoint: /kaggle/working/mistral_nemo_instruct_2407_4bit_bnb


`torch_dtype` is deprecated! Use `dtype` instead!
2026-01-18 16:27:00.888300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768753620.911309     829 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768753620.916249     829 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768753620.929421     829 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768753620.929435     829 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768753620.929438     829

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from sentence_transformers import SentenceTransformer
import faiss
import fitz  
import re

def extract_text_from_upload(pdf_input):
    try:
        if isinstance(pdf_input, str):
            # If it's a string, it's a path
            doc = fitz.open(pdf_input)
        else:
            # If it's not a string, it's bytes from the streamlit upload
            doc = fitz.open(stream=pdf_input, filetype="pdf")
            
        full_text = []
        for page in doc:
            text = page.get_text("text")
            text = re.sub(r'(?<=\b\w) (?=\w\b)', '', text) 
            text = re.sub(r'\s+', ' ', text)
            full_text.append(text)
        doc.close()
        return "\n".join(full_text)
    except Exception as e:
        print(f"Failed to parse PDF: {e}")
        return ""

In [10]:
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [11]:
def embed_chunks(chunks, model_name='sentence-transformers/all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks, convert_to_numpy=True)
    return model, embeddings

In [12]:
def create_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

In [13]:
def search_index(query, model, index, chunks, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

In [14]:
pdf_path = "/kaggle/input/aibook/Hands-on Machine Learning with Python Implement Neural Network Solutions with Scikit-learn and PyTorch (Ashwin Pajankar  Aditya Joshi) (Z-Library).pdf"  

text = extract_text_from_upload(pdf_path)
chunks = chunk_text(text,chunk_size=150, overlap=15)

model_embeddings, embeddings = embed_chunks(chunks)


index = create_faiss_index(embeddings)

In [15]:
question1 = "What is Regression?"
top_chunks = search_index(question1, model_embeddings, index, chunks, k=3)

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---\n{chunk}")


--- Chunk 1 ---
system that takes input text and learns how to classify the given text as “positive,” “negative,” or “neutral.” If you want to build a system to predict movie rating, you will probably collect movie descriptions, tags, genre information and other tags, and the final rating (out of ten), which will act as a label. Thus, the labels that are assigned are limited and clearly discreet and distinct. In some cases, you can have more than one label to be assigned to one data item – these are called multiclass classification problems. Regression Regression is a supervised learning technique that tries to capture the relationship between two variables. We often have one or more independent variables, or the variables that we would always know, and we want to learn how to predict the value of a dependent variable. In regression problems, we want to predict a continuous real value; that means

--- Chunk 2 ---
variable can be seen in the graph shown in Figure 7-1. Here, we are tryi

In [10]:
chunk = top_chunks[0]

prompt = f"Answer the next question: {question1} by reading the following text:{chunk}"

In [11]:
answer1 = generate_text(prompt, temperature=0.4)


--- Generation (9.57s) ---
Answer the next question: What is Regression? by reading the following text:system that takes input text and learns how to classify the given text as “positive,” “negative,” or “neutral.” If you want to build a system to predict movie rating, you will probably collect movie descriptions, tags, genre information and other tags, and the final rating (out of ten), which will act as a label. Thus, the labels that are assigned are limited and clearly discreet and distinct. In some cases, you can have more than one label to be assigned to one data item – these are called multiclass classification problems. Regression Regression is a supervised learning technique that tries to capture the relationship between two variables. We often have one or more independent variables, or the variables that we would always know, and we want to learn how to predict the value of a dependent variable. In regression problems, we want to predict a continuous real value; that means th

In [12]:
question2 = "What are the types of regression?"
top_chunks = search_index(question2, model_embeddings, index, chunks, k=3)

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---\n{chunk}")


--- Chunk 1 ---
system that takes input text and learns how to classify the given text as “positive,” “negative,” or “neutral.” If you want to build a system to predict movie rating, you will probably collect movie descriptions, tags, genre information and other tags, and the final rating (out of ten), which will act as a label. Thus, the labels that are assigned are limited and clearly discreet and distinct. In some cases, you can have more than one label to be assigned to one data item – these are called multiclass classification problems. Regression Regression is a supervised learning technique that tries to capture the relationship between two variables. We often have one or more independent variables, or the variables that we would always know, and we want to learn how to predict the value of a dependent variable. In regression problems, we want to predict a continuous real value; that means

--- Chunk 2 ---
variable can be seen in the graph shown in Figure 7-1. Here, we are tryi

In [13]:
chunk = top_chunks[0]

prompt = f"Answer the next question: {question2} by reading the following text:{chunk}"

In [14]:
answer2 = generate_text(prompt, temperature=0.7)


--- Generation (16.45s) ---
Answer the next question: What are the types of regression? by reading the following text:system that takes input text and learns how to classify the given text as “positive,” “negative,” or “neutral.” If you want to build a system to predict movie rating, you will probably collect movie descriptions, tags, genre information and other tags, and the final rating (out of ten), which will act as a label. Thus, the labels that are assigned are limited and clearly discreet and distinct. In some cases, you can have more than one label to be assigned to one data item – these are called multiclass classification problems. Regression Regression is a supervised learning technique that tries to capture the relationship between two variables. We often have one or more independent variables, or the variables that we would always know, and we want to learn how to predict the value of a dependent variable. In regression problems, we want to predict a continuous real value

In [16]:
NGROK_TOKEN = "31CaKcK72zQFZ5v6dgDeAWmtRTg_T8zD7FHZNe1WYBKrsNwN" #paste your NGROK Token from the website
API_KEY = "secret123"

In [18]:
from fastapi import FastAPI, Request, HTTPException, File, UploadFile, Form
import uvicorn, threading, socket, json, re, fitz, traceback
from pyngrok import ngrok, conf

app = FastAPI()

# ---  SETTINGS & STATE ---
PORT = 8000 

class PDFState:
    def __init__(self):
        self.pdf_hash = None
        self.chunks = []
        self.index = None

state = PDFState()

# ---  BACKEND LOGIC ---
@app.post("/generate")
async def gen(
    request: Request, 
    prompt: str = Form(...), 
    history: str = Form("[]"), 
    file: UploadFile = File(...)
):
    if request.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")

    try:
        pdf_bytes = await file.read()
        current_hash = hash(pdf_bytes)

        # Re-index if new file
        if state.pdf_hash != current_hash:
            text = extract_text_from_upload(pdf_bytes)
            state.chunks = chunk_text(text, chunk_size=150, overlap=15)
            embeddings = model_embeddings.encode(state.chunks, convert_to_numpy=True)
            state.index = create_faiss_index(embeddings)
            state.pdf_hash = current_hash

        # Retrieval
        query_vec = model_embeddings.encode([prompt], convert_to_numpy=True)
        _, indices = state.index.search(query_vec, k=2)
        context = " ".join([state.chunks[i] for i in indices[0]])

        chat_history = json.loads(history)
        
        # If history is empty, it's a fresh start - we can be more aggressive with speed
        is_fresh_chat = len(chat_history) == 0
        
        if is_fresh_chat:
            print("🧹 Fresh chat detected. Performance optimized.")        # Inside your gen function in Kaggle:

        history_text = ""
        for m in chat_history[-3:]:
            history_text += f"{m['role'].upper()}: {m['content']}\n"
        
        system_instructions = "You are a helpful AI assistant. Answer the question using ONLY the context provided. Do not repeat history or context."
        
        llm_prompt = f"<s>[INST] {system_instructions}\n\nCONTEXT:\n{context}\n\nHISTORY:\n{history_text}\n\nQUESTION: {prompt} \n\nAnswer:[/INST]"


        raw_response = generate_text(llm_prompt, max_new_tokens=1024, temperature=0.5)
        # Use rsplit to find the LAST occurrence of "Answer:"
        # maxsplit=1 means it only splits once, at the very last "Answer:" it finds
        if "Answer:" in raw_response:
            clean_answer = raw_response.rsplit("Answer:", 1)[-1].strip()
        elif "[/INST]" in raw_response:
            # Fallback: if "Answer:" isn't there, take everything after the instruction
            clean_answer = raw_response.split("[/INST]")[-1].strip()
        else:
            clean_answer = raw_response.strip()
        
        clean_answer = clean_answer.split("Question:")[0].strip()
        
        return {"response": clean_answer}
    except Exception as e: # <--- THIS WAS MISSING
        print(traceback.format_exc())
        return {"response": f"Internal Error: {str(e)}"}

ngrok.kill()
conf.get_default().auth_token = NGROK_TOKEN

public_url = ngrok.connect(PORT).public_url
print(f"\n🚀 NEW URL: {public_url}\n")

def run():
    uvicorn.run(app, host="0.0.0.0", port=PORT)

threading.Thread(target=run, daemon=True).start()


🚀 NEW URL: https://ungoaded-subdorsally-bethanie.ngrok-free.dev



INFO:     Started server process [829]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



--- Generation (7.37s) ---
 You are a helpful AI assistant. Answer the question using ONLY the context provided. Do not repeat history or context.

CONTEXT:
belong to the first type (Setosa), which have been correctly classified. However, there are 11 samples in the second type, out of which ten have been correctly classified, and one has been classified as the third type. Then there are 12 samples, all of which are correctly identified as the third type. sklearn.metrics also contains functions for precision, recall, and F-measure. All these functions take at least two arguments: the actual class labels and the predicted class labels. If you have more than two classes, you can give the additional function parameter for average, which may contain one of the following values: binary: Default, the function only reports the results for the positive class labels. micro: Calculate metrics globally by counting the total true positives, false negatives, and false positives. macro: Calculate m